### Example of generating data and training the TT channel

In [ ]:
from multiprocessing import cpu_count
import os
os.environ["JULIA_NUM_THREADS"] = str(cpu_count())
from juliacall import Main as jl
jl.Pkg.activate("/home/zaldivar/Documents/Androniki/Github/GWBackFinder.jl")
jl.seval("using GWBackFinder")
import torch
from sbi import utils as utils
from torch.distributions.normal import Normal
import sys
from tqdm import tqdm
import numpy as np
sys.path.append("/home/zaldivar/Documents/Androniki/Github/GWBackFinder_python")
from src.GWBackFinder import train as noise_train

#### load the prior and sample 1 million points

In [ ]:
prior_c = Normal(torch.tensor([15.]), torch.tensor([0.2*15]))
prior, *_ = utils.process_prior(prior_c)  
z=prior.sample((1000,))
#np.save("/data/users/Androniki/Dani/z_noise.npy",z)

### Define frequency range

In [ ]:
f=jl.range(3*1e-5, 0.5, step=1e-6)

### generate training data using the julia package

In [ ]:
gw_total_noise=[]
for i in tqdm(range(len(z))):
    Data_total = jl.GWBackFinder.model_noise_train_data(f, z[i].numpy()[0])
    gw_total_noise.append(np.array(Data_total))
    

### convert to tensor 

In [ ]:
gw_total_noise=torch.tensor(np.array(gw_total_noise) , dtype=torch.float32)# %%
thetas=torch.tensor(z[0:len(z)])

### check shapes

In [ ]:
print(gw_total_noise.shape)
print(thetas.shape)


### train and save inference in the file train_200_noise.pkl

In [ ]:

noise_train.train(thetas=thetas, gw_total=gw_total_noise, prior=prior, resume_training=False, validation_fraction=0.2,
          learning_rate=1e-4, show_train_summary=True, max_num_epochs=200, 
          path_saved=None,path_inference="/data/users/Androniki/", name_file="train_200_noise.pkl", model_type="nsf", hidden_features=64, num_transforms=3)

### get posterior from the train_200_noise.pkl and save it in posterior_noise.pkl

In [ ]:
noise_train.get_posterior("/data/users/Androniki/train_200_noise.pkl","posterior_noise.pkl")# %%